In [ ]:
import yaml

data = [{'name':'Eenie','fields':[{'name':'eey'},{'name':'eez'}]},
        {'name':'Meenie','fields':[{'name':'meh'}]}]

print(yaml.dump(data,sort_keys=False))

### Parsing - import

- https://pyparsing-docs.readthedocs.io/en/latest/HowToUsePyparsing.html
- https://stackoverflow.com/questions/25238740/ested-dictionary-output-from-pyparsing

In [ ]:
import pyparsing as pp
import json
from pyparsing import Word, Suppress, Group, ZeroOrMore, Keyword

data = r"""
reg Eenie { field eey; field eez; }
reg Meenie { field meh; }
"""

### Linear

In [ ]:
LPAR, RPAR, SEMI = map(Suppress, ('{};'))

g_field = Keyword('field') + pp.common.identifier + SEMI
g_reg_header = Keyword('reg') + pp.common.identifier
g_reg = g_reg_header + LPAR + ZeroOrMore(g_field) + RPAR
g_head = ZeroOrMore(g_reg)

res = g_head.parse_string(data)
print("LI", res.as_list())

### Groups

In [ ]:
LPAR, RPAR, SEMI = map(Suppress, ('{};'))

g_field = Group(Keyword('field') + pp.common.identifier('i') + SEMI)
g_reg_header = Keyword('reg') + pp.common.identifier('r')
g_reg = Group(g_reg_header + LPAR + ZeroOrMore(g_field)('f') + RPAR)
g_head = ZeroOrMore(g_reg)('h')

res = g_head.parse_string(data)
print("LI", res.as_list())
print("DI", res.as_dict())

assert res.h[0].f[1].i == 'eez'

### Named groups, suppressed

In [ ]:
LPAR, RPAR, SEMI = map(Suppress, ('{};'))

# if Group() was on `g_field` and not on `ZeroOrMore(g_field)`, it would be ['eey'], ['eez']
g_field = Suppress(Keyword('field')) + pp.common.identifier + SEMI
g_reg_header = Suppress(Keyword('reg')) + pp.common.identifier('r')
g_reg = Group(g_reg_header + LPAR + Group(ZeroOrMore(g_field))('f') + RPAR)
g_head = ZeroOrMore(g_reg)('h')

res = g_head.parse_string(data)
print("LI", res.as_list())
print("DI", res.as_dict())

assert res.h[0].f[1] == 'eez'

### Ignore syntax element


In [ ]:
ndata = r"""
reg Eenie { field eey; field eez; whatever xxx;}
reg Meenie { field meh; }
"""

LPAR, RPAR, SEMI = map(Suppress, ('{};'))

g_field = Suppress(Keyword('field')) + pp.common.identifier + SEMI
g_whatever = Keyword('whatever') + pp.common.identifier + SEMI
g_reg_header = Suppress(Keyword('reg')) + pp.common.identifier('r')
g_reg = Group(g_reg_header + LPAR + Group(ZeroOrMore(g_field|g_whatever))('f') + RPAR)
g_reg.ignore(g_whatever)
g_head = OneOrMore(g_reg)('h')

res = g_head.parse_string(ndata)
#print("DM", res.dump())
print("LI", res.as_list())
print("DI", res.as_dict())